<a href="https://colab.research.google.com/github/AyushSoni14/CS384-Attendance-Tracker/blob/main/Mid_Sem_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#false attendance count =!(6-8) ka count
#total attandace_marked=(attance between 6-8 including 3 attendance per day)
#extra attendace markded=(6-8 ke bitch me agr 2 se jyada lagayi hao to)
#extra attendace marked (both)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


def read_csv_file(file_path):
    try:
        data = pd.read_csv(file_path)
        print(f"Successfully loaded {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{file_path}' is empty.")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")


input_attendance = read_csv_file("input_attendance.csv")

if input_attendance is not None:
    try:
        print(input_attendance.head())
        input_attendance[['RollNo', 'Name']] = input_attendance['Roll'].str.split(n=1, expand=True)
        input_attendance = input_attendance.drop(columns=['Roll'])
    except KeyError:
        print("Error: 'Roll' column not found in input_attendance.")
    except Exception as e:
        print(f"Error during splitting columns: {e}")

    print(input_attendance.head())

# Function to read student list and handle spaces
def read_student_list(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        data = [line.strip().split(maxsplit=1) for line in lines if len(line.strip()) > 0]
        return pd.DataFrame(data, columns=['Roll', 'Name'])
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred while reading the student list: {e}")

output_attendance = read_student_list("stud_list.txt")


def load_class_dates(file_path):
    classes_taken_dates = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if 'classes_taken_dates' in line:
                    part = line.split('=', 1)[1].strip()
                    part = part.replace('classes_taken_dates = ', '').strip(' []"')
                    classes_taken_dates = part.split('", "')
                    print(f"Loaded class dates: {classes_taken_dates}")
                    classes_taken_dates = [date.strip('"') for date in classes_taken_dates]
        print(f"Loaded class dates: {classes_taken_dates}")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred while reading the class dates: {e}")
    return classes_taken_dates

# Load class dates from 'python dates.txt'
classes_taken_dates = load_class_dates("/content/python dates.txt")

if output_attendance is not None:
    for date in classes_taken_dates:
        output_attendance[date] = 0

    print(output_attendance.head())


def is_between_6_and_8_pm(timestamp):
    try:
        time = timestamp.time()
        start_time = datetime.strptime("18:00:00", "%H:%M:%S").time()
        end_time = datetime.strptime("20:00:00", "%H:%M:%S").time()
        return start_time <= time <= end_time
    except Exception as e:
        print(f"Error while checking timestamp: {e}")
        return False


if input_attendance is not None:
    try:
        input_attendance['Timestamp'] = pd.to_datetime(input_attendance['Timestamp'],
                                                       format="%d/%m/%Y %H:%M:%S")
    except KeyError:
        print("Error: 'Timestamp' column not found.")
    except Exception as e:
        print(f"Error during timestamp conversion: {e}")

    try:
        for date in classes_taken_dates:
            class_date = datetime.strptime(date, "%d/%m/%Y").date()
            valid_attendance = input_attendance[
                (input_attendance['Timestamp'].dt.date == class_date) &
                (input_attendance['Timestamp'].apply(is_between_6_and_8_pm))
            ]

            for roll in valid_attendance['RollNo']:
                if roll in output_attendance['Roll'].values:
                    output_attendance.loc[output_attendance['Roll'] == roll, date] += 1

        print(output_attendance.head())
    except Exception as e:
        print(f"Error during attendance update: {e}")

try:
    invalid_attendance = input_attendance[~input_attendance['Timestamp'].apply(is_between_6_and_8_pm)]
    false_attendance_counts = invalid_attendance['RollNo'].value_counts()

    output_attendance['False Attendance Count'] = output_attendance['Roll'].map(
        false_attendance_counts).fillna(0).astype(int)

    false_attendance_df = false_attendance_counts[false_attendance_counts > 0].reset_index()
    false_attendance_df.columns = ['RollNo', 'False Attendance Count']
    result_df = false_attendance_df.merge(output_attendance[['Roll', 'Name']],
                                          left_on='RollNo', right_on='Roll', how='left')
    print(result_df[['RollNo', 'Name', 'False Attendance Count']])
except Exception as e:
    print(f"Error while handling false attendance: {e}")

try:
    output_attendance.to_csv("output_attendance.csv", index=False)
except Exception as e:
    print(f"Error while saving output attendance to CSV: {e}")

output_attendance_file = read_csv_file("output_attendance.csv")

if output_attendance_file is not None:
    try:
        output_attendance_file['Total Count Dates'] = output_attendance_file.iloc[:, 2:9].count(axis=1)
        output_attendance_file['Total Attendance_Marked'] = output_attendance_file.iloc[:, 2:9].sum(axis=1)
        output_attendance_file['Total Attendance_Marked_Including_False_Attendance'] = output_attendance_file.iloc[:, 2:10].sum(axis=1)
        output_attendance_file['Total Attendance Allowed'] = output_attendance_file['Total Count Dates'] * 2

        output_attendance_file['Extra Attendance Marked(Excluding False Attendance)'] = (
            output_attendance_file.iloc[:, 2:9] > 2).sum(axis=1)

        output_attendance_file['Extra Attendance Marked(Including False Attendance)'] = (
            output_attendance_file['Extra Attendance Marked(Excluding False Attendance)'] +
            output_attendance_file['False Attendance Count'])

        output_attendance_file['Net_Percentage_attendance'] = (
            (output_attendance_file['Total Attendance_Marked'] -
             output_attendance_file['Extra Attendance Marked(Excluding False Attendance)']) /
            output_attendance_file['Total Attendance Allowed']) * 100

        print(output_attendance_file.head())
    except Exception as e:
        print(f"Error while processing attendance data: {e}")

try:
    def highlight_attendance(value):
        if value > 2:
            return 'background-color: #FF0000'
        elif value == 1:
            return 'background-color: #FFFF00'
        elif value == 2:
            return 'background-color: #00FF00'
        else:
            return ''

    styled_df = output_attendance_file.style.applymap(highlight_attendance, subset=output_attendance_file.columns[2:9])
    styled_df.to_excel('styled_attendance.xlsx', engine='openpyxl', index=False)
    print("Styled attendance saved to 'styled_attendance.xlsx'.")
except Exception as e:
    print(f"Error while saving styled attendance: {e}")


Successfully loaded input_attendance.csv
             Timestamp                           Roll
0  06/08/2024 18:18:23           2201MM26 Rudra Goyal
1  06/08/2024 18:18:26  2201CE30 Lakshya Pratap Singh
2  06/08/2024 18:18:26       2201CB19 Chhavi Bamoriya
3  06/08/2024 18:18:28          2201CE31 Mayank Kumar
4  06/08/2024 18:18:31         2201CB35 Nakka Supraja
             Timestamp    RollNo                  Name
0  06/08/2024 18:18:23  2201MM26           Rudra Goyal
1  06/08/2024 18:18:26  2201CE30  Lakshya Pratap Singh
2  06/08/2024 18:18:26  2201CB19       Chhavi Bamoriya
3  06/08/2024 18:18:28  2201CE31          Mayank Kumar
4  06/08/2024 18:18:31  2201CB35         Nakka Supraja
Loaded class dates: ['06/08/2024', '13/08/2024', '20/08/2024', '27/08/2024', '03/09/2024', '17/09/2024', '01/10/2024']
Loaded class dates: ['06/08/2024', '13/08/2024', '20/08/2024', '27/08/2024', '03/09/2024', '17/09/2024', '01/10/2024']
       Roll             Name  06/08/2024  13/08/2024  20/08/2024  2

<ipython-input-14-f16cb6a422a1>:174: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = output_attendance_file.style.applymap(highlight_attendance, subset=output_attendance_file.columns[2:9])
